In [ ]:
from pymystem3 import Mystem
from pprint import pprint
import logging
import re

## Настройка логгера

In [ ]:
logging.basicConfig(filename='preparation_results.log',
                    format='[%(asctime)s] [%(levelname)s] %(message)s',
                    level=logging.DEBUG)

lg = logging.getLogger("L")
lg.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
ch.setFormatter(formatter)
lg.addHandler(ch)

## Загрузка стеммера

In [ ]:
lg.info("Loading mystem")
m = Mystem()
lg.info("Loaded mystem")

## Вспомогательные функции

In [ ]:
def parse_gr(gr):
    options = re.search('\(([^\)]*)\)', gr, re.IGNORECASE)

    if options:
        title = options.group(1)
        for stuff in title.split('|'):
            yield gr.replace("(" + title + ")", stuff)
    else:
        yield gr

## Запись пар с их грамматической информацией

Создаем множество кортежей типа  
(грамматическое описание, начальная форма, слово)

In [ ]:
lines = set([])

with open("data/text.txt", "rt") as input_file:
    lg.info("file opened")

    # i = 0
    for line in input_file:
        # i += 1
        # if i % 1000 == 0:
        #     print(i)
        
        for w in m.analyze(line):
            if 'analysis' in w:
                for item in w['analysis']:
                    for gramm_info in parse_gr(item['gr']):
                        lines.add("\t".join(
                            [gramm_info, item['lex'], w['text'].lower()]) + "\n")
                        # lines.add("\t".join(
                        #     [gramm_info, item['lex'], w['text'].lower()]).encode("utf-8") + "\n")


Записываем их в файл pairs_with_grammar

In [ ]:
with open("data/pairs_with_grammar.tsv", "w") as f:
    for line in lines:
        f.write(line)

## Запись данных для обучения

Читаем файл

In [ ]:
dict = {}

for line in open("data/pairs_with_grammar.tsv", "r+"):
    if line.strip():
        desc, normal, form = line.strip().split("\t")
        if desc not in dict:
            dict[desc] = []
        dict[desc].append((normal, form))

logging.info("Pairs acquired")

И создаем декартово произведение пар (слово, нач. форма)

In [ ]:
writer = open("data/relations_pairs.tsv", "w+")

for desc in dict:
    for p0 in dict[desc]:
        for p1 in dict[desc]:
            if not p0 == p1:
                writer.write("\t".join([p0[0], p0[1], p1[0], p1[1]]) + "\n")

writer.close()
lg.info("Relations pairs acquired")